In [ ]:
# Installing and Updating Libraries
!pip install segmentation_models
!pip install --upgrade tensorflow
!pip install --upgrade keras

In [ ]:
# Importing Libraries
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
# Loading Images
images = []
for directory_path in glob("/content/drive/MyDrive/Dataset Address/Dataset/images/*"):
    for img_path in glob(os.path.join(directory_path, "*.jpg")) or glob(os.path.join(directory_path, "*.jpeg")):
        img=tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
        img=np.asarray(img) 
        images.append(img)

In [ ]:
# Loading Masks
masks = []
for directory_path in glob("/content/drive/MyDrive/Dataset Address/Dataset/masks/*"):
    for mask_path in glob(os.path.join(directory_path, "*.jpg")) or glob(os.path.join(directory_path, "*.jpeg")):
        mask=tf.keras.preprocessing.image.load_img(mask_path, target_size=(256, 256))
        mask=np.asarray(img) 
        masks.append(mask)

In [2]:
# Creating the U-Net Model
BACKBONE = 'vgg16'
preprocess_input = sm.get_preprocessing(BACKBONE)

x_train, x_test, y_train, y_test = train_test_split(images, masks, test_size=0.2)

model = sm.Unet(BACKBONE, encoder_weights='imagenet')

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])

print(model.summary())

history=model.fit(x_train, 
          y_train,
          batch_size=32, 
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
# Saving the Model
model.save("/content/drive/MyDrive/model_segmentation.h5")

In [ ]:
# Training and Test Loss Curves
loss = history.history['loss']
test_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, test_loss, 'r', label='Test loss')
plt.title('Training and test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Precition Using the Model
model = tf.keras.models.load_model("/content/drive/MyDrive/model_segmentation.h5", compile=False)

test_img = cv2.imread('Example Image.jpg', cv2.IMREAD_COLOR)       
test_img = cv2.resize(test_img, (256, 256))
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
test_img = np.expand_dims(test_img, axis=0)

prediction = model.predict(test_img)

prediction_image = prediction.reshape((256, 256))
plt.imshow(prediction_image, cmap='gray')
plt.imsave('segmented_image.jpg', prediction_image, cmap='gray')